# H&M 고객 데이터 — 전처리(판다스) 4회차 문제


이번 회차는 **머신러닝(분류)** 관점 전처리입니다.  
목표는 `Active`(0/1)를 예측할 수 있도록 모델이 먹을 수 있는 X(피처 행렬)을 만드는 것!

오늘의 핵심 루틴  
- **Split -> Fit -> Transform** (누수 방지)  
- 결측치 + 결측 플래그  
- 희귀 범주 묶기(Rare)  
- 빈도 기반 파생변수(=Frequency Encoding)  
- 원-핫 인코딩 + 컬럼 정렬(reindex)  
- 스케일링(StandardScaler)  

> 데이터가 100만 행이라 노트북이 버벅일 수 있어요.  
> 실습은 아래에서 **샘플링(예: 200,000행)** 후 진행하고, 익숙해지면 샘플 줄을 주석 처리하고 전체로 돌려봅시다.


In [56]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

df = pd.read_csv("customer_hm.csv")

# 실습 속도 때문에 샘플링 (필요 없으면 주석 처리!)
# df = df.sample(200_000, random_state=42).reset_index(drop=True)

print(df.shape)
df.head()


(1048575, 6)


,customer_id,FN,Active,club_member_status,fashion_news_frequency,age
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0,0,ACTIVE,NONE,49
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0,0,ACTIVE,NONE,25
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0,0,ACTIVE,NONE,24
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0,0,ACTIVE,NONE,54
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1,1,ACTIVE,Regularly,52


---

### 문제 1 — SPLIT: `Active`를 y로 두고 Train/Test 나누기
- `y = df['Active']`
- `X = df.drop(columns=['Active'])`
- `train_test_split`로 **train/test**를 나누세요.
  - `test_size=0.2`, `random_state=42`, `stratify=y`
- `X_train`, `X_test`, `y_train`, `y_test` 만들고, **각각 shape 출력**


In [57]:
# TODO: 여기에서 풀이 코드를 작성하세요.
X = df.drop(columns=['Active'])
y = df['Active']

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,  #국룰 
    random_state=42,    #
    stratify=y  #stat을 공평하게 유지하기 위해?
)

In [58]:
X_train.shape   #규칙은 X_train만 / 훈련 점수가 당연히 높음 실제보다(정확도가)
# 왜 X 로 쓰는가? 관례~ 네이밍 약속
# y는 샘플만 찍으니까(타겟이 하나니까)

(838860, 5)

In [59]:
y_train.shape

(838860,)

In [60]:
X_test.shape

(209715, 5)

In [61]:
y_test.shape

(209715,)

In [62]:
df.shape

(1048575, 6)

---

### 문제 2 — 결측 플래그 + 최빈값 채우기: `fashion_news_frequency`
`fashion_news_frequency`에 결측치가 아주 조금 있습니다.

1) 결측 플래그 만들기
- `fashion_news_frequency_isna` 컬럼을 만들고 (결측이면 1, 아니면 0)
- train/test 둘 다에 붙이세요.

2) 결측 채우기 (누수 금지!)
- **train에서만** 최빈값(mode)을 구해서
- train/test에 **동일한 값으로** 결측을 채우세요.

> 힌트: `mode()[0]`


In [63]:
# TODO: 여기에서 풀이 코드를 작성하세요.
col = 'fashion_news_frequency'
#1 결측 플래그 만들기
X_train[f"{col}_isna"] = X_train[col].isna().astype(int)
X_test[f"{col}_isna"] = X_test[col].isna().astype(int)
#2. 결측 채우기
fn_mode = X_train[col].mode(dropna=True)[0]  #결측치가 젤 많으면 제외하려고 dropna=True 붙임

X_train[col] = X_train[col].fillna(fn_mode)
X_test[col] = X_test[col].fillna(fn_mode)

#3 결과 출력
X_train[[col, f"{col}_isna"]].head()

,fashion_news_frequency,fashion_news_frequency_isna
1000717,NONE,0
248897,Regularly,0
23861,NONE,0
775740,Regularly,0
935733,Regularly,0


In [64]:
# #내가 푼거

# for col in ['fashion_news_frequency']:     #맞네 하나니까 이렇게 쓸 필요가 없구나
#     X_train[f"{col}_isna"] = X_train[col].isna().astype(int)    #{}도 [col]로 변경
#     X_test[f"{col}_isna"] = X_test[col].isna().astype(int)

In [65]:
# fashion_med_table = (
#     X_train.groupby(['fashion_news_frequency'].mode()[0].reset_index().rename(columns={'fashion_news_frequency':'fs_new_fill'}))
# )

In [66]:
# X_train = X_train.merge(fs_new_fill, on=["Pclass","Sex"], how="left")
# X_train["Age"] = X_train["Age"].fillna(X_train["Age_med"])
# X_train = X_train.drop(columns=["Age_med"])

# X_test = X_test.merge(fs_new_fill, on=["Pclass","Sex"], how="left")
# X_test["Age"] = X_test["Age"].fillna(X_test["Age_med"])
# X_test = X_test.drop(columns=["Age_med"])

---

### 문제 3 — 희귀 범주 묶기(Rare): `fashion_news_frequency`
`fashion_news_frequency`는 대부분이 `NONE`/`Regularly`인데, `Monthly`처럼 **아주 작은 범주**가 있습니다.

- train에서 `value_counts()`로 빈도를 구하고
- **빈도 < 1,000** 인 범주는 `Rare`로 묶으세요.
- 같은 규칙을 test에도 적용하세요.

> 기준(1,000 미만)은 **train에서만 정하기!**


In [67]:
# TODO: 여기에서 풀이 코드를 작성하세요.

#범주형 컬럼 지정....은 안해도 되겠다         
# cat_cols = ['']

# df = pd.get_dummies(df, columns=['fashion_news_frequency'], drop_first=True)

# df.head()

In [68]:

# 특정 빈도의 행(index) 구하기(train만)
fn_counts = X_train[col].value_counts()
rare_fn = fn_counts[fn_counts < 1000].index

# replace를 통해 rare 대ㅔ하기 (적용은 둘다)

X_train[col] = X_train[col].replace(rare_fn,"Rare")
X_test[col]  = X_test[col].replace(rare_fn,"Rare")

In [69]:
X_train[col].info()

<class 'pandas.core.series.Series'>
Index: 838860 entries, 1000717 to 814467
Series name: fashion_news_frequency
Non-Null Count   Dtype 
--------------   ----- 
838860 non-null  object
dtypes: object(1)
memory usage: 12.8+ MB


---

### 문제 4 — 빈도 기반 파생변수(Frequency Encoding)
이번에는 `TicketFreq` 같은 느낌으로, 범주 자체를 **빈도 숫자**로 바꿔봅시다.

1) `club_member_status` 빈도
- train에서 `value_counts()`로 빈도표 만들기
- `status_freq` 컬럼을 만들어서 train/test에 붙이기 (`map`)
- test에서 모르는 범주가 나오면 `0`으로 처리 (`fillna(0)`)

2) `fashion_news_frequency` 빈도
- 위와 똑같이 `fn_freq` 컬럼을 만들어서 train/test에 붙이기

> 포인트: `fit(빈도표 만들기)`은 train, `transform(map)`은 train/test 둘 다!


In [70]:
# TODO: 여기에서 풀이 코드를 작성하세요.
## 
status_count = X_train['club_member_status'].value_counts()
X_train['status_freq'] = X_train['club_member_status'].map(status_count).fillna(0)    #status_freq` 컬럼을 만들어서 train/test에 붙이기 (`map`
X_test['status_freq'] = X_test['club_member_status'].map(status_count).fillna(0)


fn_count = X_train['fashion_news_frequency'].value_counts()
X_train['fn_freq'] = X_train['fashion_news_frequency'].map(fn_count).fillna(0)
X_test['fn_freq'] = X_test['fashion_news_frequency'].map(fn_count).fillna(0)

In [71]:
## 이것이 빈도표여 
# #범주 자체를 빈도 숫자로 바꿔준겨

X_train[['club_member_status', 'status_freq', 'fashion_news_frequency', 'fn_freq']].head()

,club_member_status,status_freq,fashion_news_frequency,fn_freq
1000717,PRE-CREATE,52372,NONE,539708
248897,ACTIVE,786199,Regularly,298628
23861,ACTIVE,786199,NONE,539708
775740,ACTIVE,786199,Regularly,298628
935733,ACTIVE,786199,Regularly,298628


---

### 문제 5 — 파생변수: 나이 구간 + 뉴스 수신 여부
모델이 패턴을 잡기 쉽게, 아주 가벼운 파생변수를 몇 개 만들어봅시다.

1) `age_band`
- 아래 구간으로 `pd.cut`해서 `age_band`를 만드세요.
  - bins: `[0, 19, 29, 39, 49, 59, 120]`
  - labels: `['10대', '20대', '30대', '40대', '50대', '60대+']`

2) `is_senior`
- 나이가 60 이상이면 1, 아니면 0

3) `news_opt_in`
- `fashion_news_frequency != 'NONE'` 이면 1, 아니면 0

> train/test 둘 다에 만들기!


In [72]:
# TODO: 여기에서 풀이 코드를 작성하세요.
#나이 파생변수 만들기
bins = [0, 19, 29, 39, 49, 59, 120]
labels = ['10대', '20대', '30대', '40대', '50대', '60대+']

X_train['age_band'] = pd.cut(df['age'], bins=bins, labels=labels, right=True)
# 적용 둘다해야함
X_test['age_band'] = pd.cut(df['age'], bins=bins, labels=labels, right=True)

In [73]:
# 시니어입니까?
# 나이가 60 이상이면 1, 아니면 0
# 불.시로 만든다음에 넣는건가?

X_train["is_senior"] = (X_train['age'] >= 60).astype(int).value_counts()
X_test["is_senior"] = (X_test['age'] >= 60).astype(int).value_counts()

In [74]:
X_train["news_opt_in"] = (X_train['fashion_news_frequency'] != 'NONE').astype(int).value_counts()
X_test["news_opt_in"] = (X_test['fashion_news_frequency'] != 'NONE').astype(int).value_counts()

In [75]:
X_train[['age_band', 'is_senior', "news_opt_in"]] 

,age_band,is_senior,news_opt_in
1000717,30대,NaN,NaN
248897,20대,NaN,NaN
23861,40대,NaN,NaN
775740,30대,NaN,NaN
935733,20대,NaN,NaN
...,...,...,...
648158,20대,NaN,NaN
32893,40대,NaN,NaN
675353,20대,NaN,NaN
431003,20대,NaN,NaN


---

### 문제 6 — 이상치 캡핑 + 로그 변환(분포 완화)
이번 회차의 포인트 중 하나!

1) `age` IQR 캡핑
- train의 `age`로 Q1/Q3/IQR 계산해서 `lo/hi` 만들기
- `age_cap = age.clip(lo, hi)` 를 train/test에 만들기

2) `log1p` 변환
- `age_log1p = np.log1p(age_cap)` 만들기
- `status_freq_log1p = np.log1p(status_freq)`
- `fn_freq_log1p = np.log1p(fn_freq)`

> 캡핑 경계(lo/hi)도 **train에서만 계산**합니다.


In [76]:
# TODO: 여기에서 풀이 코드를 작성하세요.
q1, q3 = X_train["age"].quantile([0.25, 0.75])
iqr = q3 - q1
lo, hi = q1 - 1.5 * iqr, q3 + 1.5 * iqr

#1
X_train['age_cap'] = X_train['age'].clip(lo, hi)    #쓸때마다 clipstudio같네...
X_test['age_cap'] = X_test['age'].clip(lo, hi)

#2
X_train['age_log1p'] = np.log1p(X_train['age_cap']).clip(lower=0)     #log1p
X_test['age_log1p'] = np.log1p(X_test['age_cap'])           #lower=0 너 뭐더라


X_train['status_freq_log1p'] = np.log1p(X_train['status_freq'])
X_test['status_freq_log1p'] = np.log1p(X_test['status_freq'])


X_train['fn_freq_log1p'] = np.log1p(X_train['fn_freq'])
X_test['fn_freq_log1p'] = np.log1p(X_test['fn_freq'])

#X_train하되, iqr 방식으로 lw, hi로 경계값 뽑고  train 기준으로 뽑은 그 경계값을 갖다가 클리핑(캡핑)하는 것
# 이상치 중 낮은 애들은 lo에 높은 애들은 hi애 넣겠다
# 

- 에엣 왤ㅋㅔ 커졋어

- `.clip(lower=0)` 너가 왜 들어가야하는지? 나는 모르겟솨.
    - 정규분포를 
- 로그변환 왜 하느냐 : 분포를 골고루 하게 하기 위해서 -> 정규분포 모양에 비슷하게 만들기 위해서 (학습 잘되라고) 무조건 해야하는 거 아님. (선택! 지금은 문제니까 하는 것)
- 

In [77]:
X_test[['age', 'age_cap', 'age_log1p', 'status_freq', 'status_freq_log1p', 'fn_freq','fn_freq_log1p']]

,age,age_cap,age_log1p,status_freq,status_freq_log1p,fn_freq,fn_freq_log1p
610169,43,43.0,3.784190,786199,13.574966,539708,13.198785
628060,22,22.0,3.135494,786199,13.574966,539708,13.198785
758948,51,51.0,3.951244,786199,13.574966,539708,13.198785
432191,23,23.0,3.178054,786199,13.574966,298628,12.606957
437285,31,31.0,3.465736,786199,13.574966,539708,13.198785
...,...,...,...,...,...,...,...
290009,21,21.0,3.091042,786199,13.574966,539708,13.198785
208197,30,30.0,3.433987,786199,13.574966,539708,13.198785
644198,40,40.0,3.713572,786199,13.574966,539708,13.198785
410298,56,56.0,4.043051,786199,13.574966,298628,12.606957


---

### 문제 7 — 원-핫 인코딩(get_dummies) + 컬럼 정렬(reindex)
이제 모델이 먹을 수 있게 범주형을 원-핫 인코딩합니다.

- 원-핫 대상: `club_member_status`, `fashion_news_frequency`, `age_band`
- `pd.get_dummies(..., drop_first=True)`로 train/test 각각 변환
- **가장 중요한 것!** train 컬럼 기준으로 test 컬럼 정렬:

```python
X_test_dum = X_test_dum.reindex(columns=X_train_dum.columns, fill_value=0)
```

- 변환 후 `shape` 확인


In [78]:
# TODO: 여기에서 풀이 코드를 작성하세요.
# "customer_id", "age", "age_cap", "status_freq", "fn_freq" 해당 컬럼은 모델 입력에서 빼자! drop!
# 순서 ㅇ 0 1 이 있으니까

no_col = ["customer_id", "age", "age_cap", "status_freq", "fn_freq"]
cat_col = ["club_member_status", "fashion_news_frequency", "age_band"]
X_train_model = X_train.drop(columns=no_col)
X_test_model = X_test.drop(columns=no_col)

# 원 핫 인코딩

X_train_dum = pd.get_dummies(X_train_model, columns=cat_col, drop_first=True)
X_test_dum = pd.get_dummies(X_test_model, columns=cat_col, drop_first=True)

# 아 맞네 이게? 위에서 잘못돼서 안된거였네 ㅎㅎ;;
# reindex (컬럼 정렬)
X_test_dum = X_test_dum.reindex(columns=X_train_dum.columns, fill_value=0)



In [79]:
X_train_model.head()

,FN,club_member_status,fashion_news_frequency,fashion_news_frequency_isna,age_band,is_senior,news_opt_in,age_log1p,status_freq_log1p,fn_freq_log1p
1000717,0,PRE-CREATE,NONE,0,30대,NaN,NaN,3.433987,10.866146,13.198785
248897,1,ACTIVE,Regularly,0,20대,NaN,NaN,3.258097,13.574966,12.606957
23861,0,ACTIVE,NONE,0,40대,NaN,NaN,3.806662,13.574966,13.198785
775740,1,ACTIVE,Regularly,0,30대,NaN,NaN,3.526361,13.574966,12.606957
935733,1,ACTIVE,Regularly,0,20대,NaN,NaN,3.218876,13.574966,12.606957


In [80]:
X_train_dum.info()

<class 'pandas.core.frame.DataFrame'>
Index: 838860 entries, 1000717 to 814467
Data columns (total 16 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   FN                                838860 non-null  int64  
 1   fashion_news_frequency_isna       838860 non-null  int64  
 2   is_senior                         2 non-null       float64
 3   news_opt_in                       2 non-null       float64
 4   age_log1p                         838860 non-null  float64
 5   status_freq_log1p                 838860 non-null  float64
 6   fn_freq_log1p                     838860 non-null  float64
 7   club_member_status_LEFT CLUB      838860 non-null  bool   
 8   club_member_status_PRE-CREATE     838860 non-null  bool   
 9   fashion_news_frequency_Rare       838860 non-null  bool   
 10  fashion_news_frequency_Regularly  838860 non-null  bool   
 11  age_band_20대                      838860 non-null  

In [81]:
X_test_dum.info()

<class 'pandas.core.frame.DataFrame'>
Index: 209715 entries, 610169 to 332191
Data columns (total 16 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   FN                                209715 non-null  int64  
 1   fashion_news_frequency_isna       209715 non-null  int64  
 2   is_senior                         0 non-null       float64
 3   news_opt_in                       0 non-null       float64
 4   age_log1p                         209715 non-null  float64
 5   status_freq_log1p                 209715 non-null  float64
 6   fn_freq_log1p                     209715 non-null  float64
 7   club_member_status_LEFT CLUB      209715 non-null  bool   
 8   club_member_status_PRE-CREATE     209715 non-null  bool   
 9   fashion_news_frequency_Rare       209715 non-null  bool   
 10  fashion_news_frequency_Regularly  209715 non-null  bool   
 11  age_band_20대                      209715 non-null  b

매칭이 참 잘도ㅒㅆ구나

In [82]:
X_train_model.info()

<class 'pandas.core.frame.DataFrame'>
Index: 838860 entries, 1000717 to 814467
Data columns (total 10 columns):
 #   Column                       Non-Null Count   Dtype   
---  ------                       --------------   -----   
 0   FN                           838860 non-null  int64   
 1   club_member_status           838860 non-null  object  
 2   fashion_news_frequency       838860 non-null  object  
 3   fashion_news_frequency_isna  838860 non-null  int64   
 4   age_band                     838860 non-null  category
 5   is_senior                    2 non-null       float64 
 6   news_opt_in                  2 non-null       float64 
 7   age_log1p                    838860 non-null  float64 
 8   status_freq_log1p            838860 non-null  float64 
 9   fn_freq_log1p                838860 non-null  float64 
dtypes: category(1), float64(5), int64(2), object(2)
memory usage: 64.8+ MB


---

### 문제 8 — 스케일링(StandardScaler): train만 fit!
스케일링이 필요한 수치형 컬럼만 골라서 표준화해봅시다.

- 추천 스케일링 대상
  - `FN`
  - `age_log1p`
  - `status_freq_log1p`
  - `fn_freq_log1p`

- `StandardScaler`를 준비하고
  - train: `fit_transform`
  - test: `transform`

> 결과가 맞으면 train의 평균이 0 근처로 나옵니다(완전 0은 아니어도 OK).


In [84]:
# TODO: 여기에서 풀이 코드를 작성하세요.
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
num_col = ['FN', 'age_log1p', 'status_freq_log1p', 'fn_freq_log1p'] # 숫치형

# StandardScaler 왜 너가 안뜨지 설마 아하
scale_col = StandardScaler()
X_train_dum[["FN_std", "age_log1p_std", "status_freq_log1p_std", "fn_freq_log1p_std"]] = scale_col.fit_transform(X_train_dum[num_col])
# X_train는 뒤에서 fit_transform까지 같이한다. X-test는 걍적용

X_test_dum[["FN_std", "age_log1p_std", "status_freq_log1p_std", "fn_freq_log1p_std"]] = scale_col.transform(X_test_dum[num_col])

In [85]:
### 이렇게 하시던데.... 나만 저렇게 길게 복붙했군... 
# X_train_dum[scale_cols] = scaler.fit_transform(X_train_dum[scale_cols])
# X_test_dum[scale_cols] = scaler.transform(X_test_dum[scale_cols])

In [86]:
X_train_dum[["FN_std", "age_log1p_std", "status_freq_log1p_std", "fn_freq_log1p_std"]].describe()

,FN_std,age_log1p_std,status_freq_log1p_std,fn_freq_log1p_std
count,8.388600e+05,8.388600e+05,8.388600e+05,8.388600e+05
mean,-9.032768e-17,-1.534825e-16,-3.237532e-15,-3.255600e-15
std,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00
min,-7.428174e-01,-1.901162e+00,-1.152664e+01,-2.040199e+01
25%,-7.428174e-01,-8.773766e-01,2.561349e-01,-1.143934e+00
50%,-7.428174e-01,-1.403716e-01,2.561349e-01,6.527631e-01
75%,1.346226e+00,9.626614e-01,2.561349e-01,6.527631e-01
max,1.346226e+00,2.448225e+00,2.561349e-01,6.527631e-01


In [87]:
X_train_dum[["FN_std", "age_log1p_std", "status_freq_log1p_std", "fn_freq_log1p_std"]].head()

,FN_std,age_log1p_std,status_freq_log1p_std,fn_freq_log1p_std
1000717,-0.742817,-0.306339,-3.781444,0.652763
248897,1.346226,-0.773261,0.256135,-1.143934
23861,-0.742817,0.682970,0.256135,0.652763
775740,1.346226,-0.061124,0.256135,-1.143934
935733,1.346226,-0.877377,0.256135,-1.143934


- -3.781444 너이자식 뭐야 검거해
    - 참고로 왼쪽으로 3.78만큼 찌그러져(?) 있다는 뜻
    - 아ㅓ dum으로 안해서 그러네 ㅎ헤헤;;;
    - 아니네 너 뭐냐 너
- status_freq_log1p_std : 8.388600e+05	/ -1.152664e+01	
    - 흠...
- status_freq_log1p_std : 2.561349e-01이 반복됨
    - 굉장히 같은 애들이 많다~ (25% )
- fn_freq_log1p_std : -2.040199e+01
    - 흠... 설마 내가 잘못 해온걸까?................ 샘플링 할걸그랫나 원본데이터로 하지 ㅏㅁㄹ고?..,,
    - 아니네 바꿔서 해봐도 쟤들이 튀는 거 맞네
    - 휴 내가 잘못한 게 아니라 데이터가 잘못된거였네 휴 ^^;; (사실 다행이 아님)

- zzzz X_test_dum에 집어넣을 때 train을 잘못 복붙해서 ㅎㅎ;; 잘못나온 거였네 그치 상식적으로 이건 좀 아니긴 했지 ㅎㅎ;; 휴휴 문제아들이 하나둘 보이지만? 그래도 습... 흠.. 그래도는 그래도 좀 상당히 찌그러져있는듯

---

### 문제 9 — (맛보기) 로지스틱 회귀로 학습/평가
전처리 결과가 진짜 모델 입력(X)으로 쓸 수 있는지 확인만 해봅시다.

- `LogisticRegression(max_iter=1000)`
- 정확도(accuracy) + ROC-AUC 둘 다 출력

> 데이터가 크면 학습이 부담될 수 있으니, 필요하면 위에서 샘플 수를 더 줄여도 됩니다.


In [92]:
# 임시로 일단 fillna 해버려

X_train_dum = X_train_dum.fillna(0)
X_test_dum = X_test_dum.fillna(0)

In [91]:
# TODO: 여기에서 풀이 코드를 작성하세요.
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

model = LogisticRegression(max_iter=1000)   # 로지스틱 회귀분석을 돌리겠다~
model.fit(X_train_dum, y_train) # 모델링 fit. -> 모델 등에서 fit은 독립변수(X)와 종속변수(y) 사이의 관계(회귀계수(대충베타기호))를 찾아냄
                            # 원래는 (스케일러에서는) 평균(뮤)와 표준편차(시그마)를 계산 (걍 기준점만 세우는 늑김) 
                            # 그래서 여기서처럼 모델에서 .fit은 피처가 커질 때 얼마나 변하는가? 해서 학습 ㄱㄱ 하는(train에서)

pred = model.predict(X_test_dum)
proba = model.predict_proba(X_test_dum)[:, 1]

print("Accuracy:", accuracy_score(y_test, pred))
print("ROC-AUC :", roc_auc_score(y_test, proba))

Accuracy: 0.9907445819326228
ROC-AUC : 0.9939193446670437


ValueError: Input X contains NaN 오류 삐용삐용
- 왜 안돼? -> 결측이 있나봄
- 설마 아까 dum 말고 걍 train에다가 한 걸까 drop을? 설마설마